Основные цели этого задания:

- Попрактиковаться в борьбе с дисбалансом классов

- Научиться заполнять пропуски в данных

- Научиться использовать категориальные признаки.

Задача: 

по различным признакам, связанным с заявкой на грант, предсказать, будет ли заявка принята. 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from summarytools import dfSummary
import numpy as np
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import f1_score, precision_recall_curve, roc_curve, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
import plotly.graph_objects as go

План решения:

1. _Загрузите данные из csv файла. Ознакомьтесь с ними, проверьте наличие пропусков, узнайте типы признаков._

In [2]:
# Загрузка данных
df = pd.read_csv('grant_data_imb.csv')
df.shape

(4113, 39)

In [3]:
dfSummary(df)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Grant.Status[int64],Mean (sd) : 0.2 (0.4)min < med < max:0.0 < 0.0 < 1.0IQR (CV) : 0.0 (0.5),2 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAABtklEQVR4nO3dQYqDQBBG4aohSNx0I4KH8RA5bA7hbSS0iyiBpGc3m2GidjLM/PC+rVbh4qEIgp5zNuC/+/jrCwC2OKyd4O61mVWF+28557lwFvjyNFR3r9u2PYUQmpLl0zRd3P1MrHjV2h21CiE0fd/PMcZlz+KU0nEYhmYcx8rMCBUvWX30m5nFGJeu664F++uCGeAbXqYggVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhgVAhYdNv0IEt3L02s6pw/JZznn86SKh4C3ev27Y9hRCakvlpmi7ufv4pVkLFu1QhhKbv+znGuOwZTCkdh2FoxnGszIxQ8ftijEvXddeC0frZQV6mIIFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIYFQIWHT11MppePexSml4/1+P5hZcPfdFwY54fF4HEpbWTvnE+dCTn21JaLfAAAAAElFTkSuQmCC"">",0(0.0%)
2,Sponsor.Code[object],1. 4D2. 2B3. 21A4. nan5. 24D6. 40D7. 34B8. 32D9. 97A10. 59C11. other,"1,006 (24.5%)915 (22.2%)375 (9.1%)257 (6.2%)114 (2.8%)91 (2.2%)81 (2.0%)73 (1.8%)46 (1.1%)44 (1.1%)1,111 (27.0%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAEeUlEQVR4nO3dO24bVwCG0SuBcKKGBGFAy/ACuIgsNovgArwPQdAUluJGKfyALTuxxMc3FH1O5yq3+DDEDO4fXTw+Pg4oXM59AH4fYiMjNjJiIyM2MmIjIzYyYiMjNjIXY4yrMcabuQ/yEx8fHx/v5z4Eh7N4+/btX8vlcj33QZ6apun24uLib8Gdj8VyuVxvNpv71Wr1MPdhvri7u/tzu92ub25u3owxxHYmFmOMsVqtHq6vrz/MfZgnruY+AIflBYGM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2Mgsxvh0f2zug3zr1M7DYSymabrdbrfrcWL3x6Zpuh1jfJz7HByODQKZC//LLCqn9GTzJDtzJ7OusqY6fyexrrKm+j2c0rrqpN6GOTwfdcmIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MicxOBl7v8+jZMZvBi4nD/XwskYvJDZ58nmScSL7Dx4MVDhpXYavBiosIt9Bi8GKryIj7pkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkdhq8GKiwi50HLwYqvJRr4WQMXsh4QSDzkp9RP5vs5dnrKmsq9vWsdZU1FYfwknWVNRV78YJARmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRG5lnrKmsqDuHZ6yprKvblWjgZ6yoyP3uyeYJxFD8MXgxbOJbvBi+GLRzTzwYvhi0chY+6ZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZL4bvBi2cEw/DF4MWzgW18LJGLyQefpk81TjaL4bvBi7cEyXXwYvm83m/nN0u/w5SPilr4OXz/82duFofNQlIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyz/pzQnAI366rLKs4KhsEMtZVZDzZyFhXkbGuImNdRcZHXTJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjIGL2QMXsi4Fk7G4IWMJxsZgxcyBi9kDF7I+KhLRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGxuCFjMELGdfCyRi8kPGCQObbn1E/oRzV13WVZRXHdrlcLtfv3r0bllUc2+UYY1xdXf0z90E4f14QyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyFyOMcb9/f0fcx+E83c5TdPt+/fvLas4OtfCyVhXkfFkI2PwQsbghYzBCxkfdcmIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MgYvJAxeCHjWjgZgxcyT/9Q2inz5H3lvg5e5j7IrxjkvH6L5XK53mw296vV6mHuw/yXu7u7P7fb7frm5ubNGENsr9RijDFWq9XD9fX1h7kP8wtXcx+A/fioS0ZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERmYxxqcrPHMf5P+c+vl4nsU0Tbfb7XY9TvwKj0HO6+daOBmDFzJeEMiIjYzYyIiNjNjIiI2M2MiIjYzYyPwLKygLjy7kd5MAAAAASUVORK5CYII="">",257(6.2%)
3,Grant.Category.Code[object],1. 10A2. 30B3. 50A4. nan5. 10B6. 20C7. 30C8. 30D9. 20A10. 30G11. other,"2,050 (49.8%)707 (17.2%)375 (9.1%)257 (6.2%)211 (5.1%)180 (4.4%)147 (3.6%)93 (2.3%)49 (1.2%)35 (0.9%)9 (0.2%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjguMCwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy81sbWrAAAACXBIWXMAAA9hAAAPYQGoP6dpAAAEcklEQVR4nO3dS04cVwCG0dutlhMmjRCSl5DMWACL8GKzCBbAMHtAiB4E2xMyiBPFMo9+VH2tNuesoIQ+3aJK9cPi6elpQGF57Avg/RAbGbGRERsZsZERGxmxkREbGbGRWYwxzsYYH459ISfm69PT0+OxL+LUrC4vLz+t1+uLY1/IKdlsNveLxeIPwe1mtV6vL66vrx/Pz88/H/tiTsHDw8OvNzc3F3d3dx/GGGLbwWqMMc7Pzz9//Pjxr2NfzAk5O/YFnCIPCGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERmY1xj/faB37Qk6Fn9X+VpvN5v7m5uZi+EZra5vN5n6M8fXY13FqbBD2Y4Owh4U/mUWlPtmcCO9Yuq6ySnrfsnWVVRL1usoT7zvmpS4ZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERuZbPBiKEI6eDEUed98Fk7G4IXMISebU4qd7D14MV5hV3sNXoxX2MchgxfjFXbipS4ZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERuZvQYvxivsY+/Bi/EKu/JZOBmDFzIeEMjscht12+

- Данные содержат 4113 записей с 39 столбцами.
- Большинство столбцов имеют тип float64.
- Есть пропущенные значения во всех столбцах, кроме таргета, что видно из количества пропущенных записей.

In [4]:
df.drop_duplicates(inplace=True)

2. _Подготовьте данные к обучению моделей:_

Отделите целевую переменную Grant.Status;

In [5]:
# Разделение данных на признаки и целевую переменную
features = df.drop(['Grant.Status'], axis=1)
target = df['Grant.Status']

Заполните пропуски

- в количественных признаках заполните пропуски средними значениями и нулями (у каждой фичи будет по два варианта),

In [6]:
# Разделяем числовые и нечисловые (категориальные) столбцы
num_cols = features.select_dtypes(include=['int64', 'float64']).columns

# Заполняем пропуски в числовых столбцах средним значением
features_mean = features.copy()
features_mean[num_cols] = features_mean[num_cols].fillna(features_mean[num_cols].mean())

# Заполняем пропуски в числовых столбцах нулями
features_zero = features.copy()
features_zero[num_cols] = features_zero[num_cols].fillna(0)

- в категориальных признаках выведите значения, которые они принимают, и найдите признак, где пропуски можно заполнить осмысленным значением. В остальных признаках заполните пропуски значением по умолчанию;

In [7]:
for column in features.select_dtypes(include=['object']).columns:
    display(f"{column}:", features[column].unique())

'Sponsor.Code:'

array(['97A', '36D', '317A', '62B', '1A', '4D', '2B', '60D', '21A',
       '149A', '252D', '65A', '40D', '24D', '34B', '166B', '6B', '29A',
       '5A', '141A', nan, '32D', '89A', '52D', '18B', '33A', '91C', '12D',
       '20D', '66B', '145A', '179C', '86B', '42B', '183C', '112D', '234B',
       '49A', '437A', '77A', '83C', '93A', '229A', '84D', '196D', '281A',
       '69A', '161A', '51C', '226B', '269A', '188D', '94B', '173A',
       '221A', '9A', '126B', '133A', '101A', '215C', '172D', '138B',
       '219C', '59C', '315C', '205A', '247C', '194B', '15C', '3C', '55C',
       '204D', '90B', '163C', '307C', '206B', '95C', '7C', '180D', '214B',
       '197A', '349A', '232D', '325A', '266B', '164D', '63C', '228D',
       '187C', '208D', '241A', '48D', '309A', '73A', '28D', '362B', '39C',
       '67C', '212D', '200D', '148D', '154B', '245A', '311C', '105A',
       '47C', '222B', '137A', '159C', '100D', '143C', '193A', '324D',
       '38B', '75C', '427C', '103C', '415C', '139C', '107C', '136

'Grant.Category.Code:'

array(['30B', '10A', '30D', '10B', '50A', '20A', '30C', nan, '20C', '30E',
       '30G', '30A', '40C', '30F'], dtype=object)

'Contract.Value.Band...see.note.A:'

...

array([nan, 'English', 'Other'], dtype=object)

'With.PHD.1:'

array(['Yes ', nan], dtype=object)

'No..of.Years.in.Uni.at.Time.of.Grant.1:'

array(['>10 to 15', 'Less than 0', '>5 to 10', '>=0 to 5', nan,
       'more than 15'], dtype=object)

In [8]:
# Получаем список категориальных признаков
categorical_cols = features.select_dtypes(include=['object']).columns

# Итерируемся по категориальным признакам и выводим информацию
for column in categorical_cols:
    # Подсчет количества непропущенных и пропущенных значений
    null_counts = features[column].isnull().sum()
    not_null_counts = features[column].notnull().sum()
    
    print(f"{column}:")
    print(f"Не пропущено: {not_null_counts}")
    print(f"Пропущено: {null_counts}")
    display(features[column].value_counts())
    print()


Sponsor.Code:
Не пропущено: 3803
Пропущено: 245


Sponsor.Code
4D      979
2B      913
21A     374
24D     114
40D      89
       ... 
308D      1
284D      1
259C      1
331C      1
225A      1
Name: count, Length: 226, dtype: int64


Grant.Category.Code:
Не пропущено: 3803
Пропущено: 245


Grant.Category.Code
10A    2020
30B     699
50A     374
10B     207
20C     177
30C     145
30D      91
20A      48
30G      34
30E       5
30A       1
40C       1
30F       1
Name: count, dtype: int64


Contract.Value.Band...see.note.A:
Не пропущено: 1931
Пропущено: 2117


...

Home.Language.1
English    312
Other       77
Name: count, dtype: int64


With.PHD.1:
Не пропущено: 2304
Пропущено: 1744


With.PHD.1
Yes     2304
Name: count, dtype: int64


No..of.Years.in.Uni.at.Time.of.Grant.1:
Не пропущено: 3313
Пропущено: 735


No..of.Years.in.Uni.at.Time.of.Grant.1
>=0 to 5        1321
>5 to 10         728
Less than 0      492
>10 to 15        413
more than 15     359
Name: count, dtype: int64

In [9]:
# Заполнение пропусков в "Sponsor.Code"
features['Sponsor.Code'].fillna('Other', inplace=True)

In [10]:
unique_codes = features['Sponsor.Code'].unique()
unique_codes.sort()
print(unique_codes)

['100D' '101A' '103C' '105A' '107C' '111C' '112D' '113A' '11C' '120D'
 '126B' '128D' '12D' '130B' '132D' '133A' '135C' '136D' '137A' '138B'
 '139C' '13A' '141A' '143C' '144D' '145A' '146B' '147C' '148D' '149A'
 '14B' '150B' '151C' '153A' '154B' '156D' '157A' '158B' '159C' '15C'
 '160D' '161A' '163C' '164D' '165A' '166B' '168D' '169A' '16D' '170B'
 '172D' '173A' '174B' '176D' '177A' '178B' '179C' '180D' '183C' '184D'
 '187C' '188D' '18B' '191C' '193A' '194B' '195C' '196D' '197A' '198B'
 '199C' '1A' '200D' '201A' '202B' '203C' '204D' '205A' '206B' '208D' '20D'
 '210B' '212D' '214B' '215C' '216D' '219C' '21A' '221A' '222B' '223C'
 '225A' '226B' '227C' '228D' '229A' '230B' '232D' '234B' '235C' '238B'
 '23C' '241A' '242B' '244D' '245A' '247C' '24D' '250B' '252D' '255C'
 '256D' '257A' '258B' '259C' '260D' '262B' '265A' '266B' '267C' '269A'
 '26B' '270B' '273A' '274B' '277A' '279C' '27C' '281A' '282B' '284D'
 '285A' '286B' '289A' '28D' '294B' '295C' '298B' '299C' '29A' '2B' '305A'
 '307C' '30

Заменили на Other

In [11]:
# Заполнение пропусков в "Grant.Category.Code" 
features['Grant.Category.Code'].fillna('20B', inplace=True)
features['Grant.Category.Code'].value_counts()

Grant.Category.Code
10A    2020
30B     699
50A     374
20B     245
10B     207
20C     177
30C     145
30D      91
20A      48
30G      34
30E       5
30A       1
40C       1
30F       1
Name: count, dtype: int64

In [12]:
# Создание матрицы корреляции Крамера
cont_table = pd.crosstab(features['Sponsor.Code'], features['Grant.Category.Code'])
n = cont_table.sum().sum()
phi_squared = (cont_table ** 2 / n).sum() / (min(cont_table.shape) - 1)
cramer_v = np.sqrt(phi_squared)

print("Коэффициент корреляции Крамера:", cramer_v)

Коэффициент корреляции Крамера: Grant.Category.Code
10A    5.635365
10B    0.343328
20A    0.071629
20B    1.068007
20C    0.316786
30A    0.004359
30B    0.735907
30C    0.357057
30D    0.149680
30E    0.011533
30F    0.004359
30G    0.061649
40C    0.004359
50A    1.630346
dtype: float64


Заполнение пропусков в "Grant.Category.Code" значением '20B' тк согласно логике его не хватает и возможно удалили.

In [13]:
# Пропуски в "Contract.Value..." заменим на 'Unknown' 
features['Contract.Value.Band...see.note.A'].fillna('Unknown', inplace=True) 
unique_codes = features['Contract.Value.Band...see.note.A'].unique()
unique_codes.sort()
display(unique_codes)

array(['A ', 'B ', 'C ', 'D ', 'E ', 'F ', 'G ', 'H ', 'I ', 'J ', 'K ',
       'L ', 'M ', 'O ', 'P ', 'Q ', 'Unknown'], dtype=object)

Видно что согласно алфавиту в последовательности не хватает 'N' но настараживает кол-во пропусков = 2117 шт так что заменим на Unknown

In [14]:
# Заполнение пропусков в "Role.1" 
features['Role.1'].fillna('OTHER', inplace=True) 
features['Role.1'].unique()

array(['CHIEF_INVESTIGATOR', 'PRINCIPAL_SUPERVISOR',
       'DELEGATED_RESEARCHER', 'EXT_CHIEF_INVESTIGATOR', 'HONVISIT',
       'OTHER', 'STUD_CHIEF_INVESTIGATOR', 'EXTERNAL_ADVISOR'],
      dtype=object)

Заменили на OTHER

In [15]:
# Заполнение пропусков в "Country.of.Birth.1"
features['Country.of.Birth.1'].fillna('Unknown', inplace=True) 
features['Country.of.Birth.1'].value_counts()

Country.of.Birth.1
Australia                 2501
Unknown                    639
Great Britain              298
Western Europe             155
Asia Pacific               149
North America              121
Eastern Europe              60
Middle East and Africa      40
The Americas                32
New Zealand                 31
South Africa                22
Name: count, dtype: int64

заменим на Unknown

In [16]:
# Заполнение пропусков в "Home.Language.1" 
features['Home.Language.1'].unique()

array([nan, 'English', 'Other'], dtype=object)

In [17]:
features['Home.Language.1'].value_counts()

Home.Language.1
English    312
Other       77
Name: count, dtype: int64

In [18]:
H_L_mapping = set(features.dropna(subset='Home.Language.1').apply(lambda f: (f['Country.of.Birth.1'].split()[0], f['Home.Language.1']), axis=1))
H_L_mapping

{('Asia', 'English'),
 ('Asia', 'Other'),
 ('Australia', 'English'),
 ('Australia', 'Other'),
 ('Eastern', 'English'),
 ('Eastern', 'Other'),
 ('Great', 'English'),
 ('Middle', 'English'),
 ('Middle', 'Other'),
 ('North', 'English'),
 ('North', 'Other'),
 ('The', 'English'),
 ('The', 'Other'),
 ('Unknown', 'English'),
 ('Unknown', 'Other'),
 ('Western', 'English'),
 ('Western', 'Other')}

Видим что в словаре для каждого континетна есть оба значения, а так же отсутствуют для New Zealand и South Africa предположим что для них Home Language будет English

In [19]:
# Обновление распределения языков для 'New Zealand' и 'South Africa'
features.loc[features['Country.of.Birth.1'] == 'New Zealand', 'Home.Language.1'] = 'English'
features.loc[features['Country.of.Birth.1'] == 'South Africa', 'Home.Language.1'] = 'English'

In [20]:
features['Home.Language.1'].value_counts()

Home.Language.1
English    365
Other       77
Name: count, dtype: int64

заменим на Unknown остальные значения

In [21]:
features['Home.Language.1'].fillna('Unknown', inplace=True) 

In [22]:
# Заполнение пропусков в "With.PHD.1"
features['With.PHD.1'] = features['With.PHD.1'].fillna('No')
features['With.PHD.1'].unique()

array(['Yes ', 'No'], dtype=object)

заменим на No остальные значения

In [23]:
# Заполнение пропусков в "No..of.Years.in.Uni.at.Time.of.Grant.1"
features['No..of.Years.in.Uni.at.Time.of.Grant.1'].value_counts(normalize=True) * 100

No..of.Years.in.Uni.at.Time.of.Grant.1
>=0 to 5        39.873227
>5 to 10        21.974042
Less than 0     14.850589
>10 to 15       12.466043
more than 15    10.836100
Name: proportion, dtype: float64

In [24]:
# Общее количество непропущенных записей
total_non_missing = features['No..of.Years.in.Uni.at.Time.of.Grant.1'].dropna().shape[0]

# Доли каждой категории
category_proportions = features['No..of.Years.in.Uni.at.Time.of.Grant.1'].value_counts() / total_non_missing

# Количество пропущенных записей
total_missing = features['No..of.Years.in.Uni.at.Time.of.Grant.1'].isnull().sum()

# Заполнение пропусков
features['No..of.Years.in.Uni.at.Time.of.Grant.1'] = features['No..of.Years.in.Uni.at.Time.of.Grant.1'].fillna(
    pd.Series(
        np.random.choice(
            ['>=0 to 5', '>5 to 10', 'Less than 0', '>10 to 15', 'more than 15'], 
            size=total_missing,
            p=category_proportions
        )
    )
)

In [25]:
print(features['No..of.Years.in.Uni.at.Time.of.Grant.1'].value_counts())

No..of.Years.in.Uni.at.Time.of.Grant.1
>=0 to 5        1384
>5 to 10         752
Less than 0      512
>10 to 15        431
more than 15     375
Name: count, dtype: int64


- в количественных признаках заполните пропуски средними значениями и нулями (у каждой фичи будет по два варианта)

- Преобразуйте категориальные признаки в количественные с помощью прямого кодирования;

- Разделите данные на обучающую и тестовую части;

In [26]:
features_ohe = pd.get_dummies(features_mean)

In [27]:
features_train, features_test, target_train, target_test = train_test_split(
    features_ohe, target, test_size=0.25, random_state=23
)
model = LogisticRegression(solver='liblinear', random_state=12, class_weight='balanced')
model.fit(features_train, target_train)
roc_auc_score(target_test, model.predict_proba(features_test)[:, 1])

0.7671149836838288

смотрим количественные признаки с NaN заполненные нулями

In [28]:
features_ohe_0 = pd.get_dummies(features_zero)

In [29]:
features_train, features_test, target_train, target_test = train_test_split(
    features_ohe_0, target, test_size=0.25, random_state=23
)
model = LogisticRegression(solver='liblinear', random_state=12, class_weight='balanced')
model.fit(features_train, target_train)
roc_auc_score(target_test, model.predict_proba(features_test)[:, 1])

0.6348860134155185

- Используйте масштабирование для получения признаков одинакового масштаба.

In [30]:
scaler = StandardScaler()
scaler.fit(features_train)
features_train_sc = scaler.transform(features_train)
features_valid_sc = scaler.transform(features_test)

In [31]:
features_train_sc, features_valid_sc, target_train, target_test = train_test_split(
    features_ohe, target, test_size=0.25, random_state=23
)
model = LogisticRegression(solver='liblinear', random_state=12, class_weight='balanced')
model.fit(features_train_sc, target_train)
roc_auc_score(target_test, model.predict_proba(features_valid_sc)[:, 1])

0.7671149836838288

In [32]:
features_train_sc, features_valid_sc, target_train, target_test = train_test_split(
    features_ohe_0, target, test_size=0.25, random_state=23
)
model = LogisticRegression(solver='liblinear', random_state=12, class_weight='balanced')
model.fit(features_train_sc, target_train)
roc_auc_score(target_test, model.predict_proba(features_valid_sc)[:, 1])

0.6348860134155185

Сравнивая эти два результата, можно сделать следующие выводы о том, что признаки, полученные из features_mean, содержат более полезную информацию для задачи классификации, чем признаки, полученные из features_zero. Масштабирование признаков не оказало существенного влияния на результат в данном случае.

3. _Изучите распределение по целевой переменной, чтобы выяснить, сбалансированы ли классы. Если классы не сбалансированы, используйте в работе хотя бы один из изученных методов борьбы с дисбалансом классов;_

In [33]:
df['Grant.Status'].value_counts()

Grant.Status
0    3209
1     839
Name: count, dtype: int64

In [34]:
def upsample(features, target, repeat=10):
    # разделяем объекты разных классов и информацию о них по разным переменным
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    # дублируем записи объектов более редкого класса
    if len(target_ones) > len(target_zeros):
        repeat = round(len(target_ones) / len(target_zeros))
        features_upsampled = pd.concat([features_ones] + [features_zeros] * repeat)
        target_upsampled = pd.concat([target_ones] + [target_zeros] * repeat)
    else:
        repeat = round(len(target_zeros) / len(target_ones))
        features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
        target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    # перемешиваем объекты
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=23)
    
    return features_upsampled, target_upsampled

In [35]:
features_train_upsampled, target_train_upsampled = upsample(features_train, target_train)

In [36]:
target_train_upsampled.value_counts()

Grant.Status
1    2460
0    2421
Name: count, dtype: int64

In [37]:
features_train_upsampled, features_test, target_train_upsampled, target_test = train_test_split(
    features_ohe, target, test_size=0.25, random_state=23
)
model = LogisticRegression(solver='liblinear', random_state=12, class_weight='balanced')
model.fit(features_train_upsampled, target_train_upsampled)
roc_auc_score(target_test, model.predict_proba(features_test)[:, 1])

0.7671149836838288

Применение upsampling к тренировочным данным не оказало существенного влияния на качество модели, так как значение ROC-AUC осталось на том же уровне

4. _Обучите модели и выберите лучшую:_

Обучите модель логистической регрессии, используя кросс-валидацию. Оцените ее качество с помощью метрики rocauc. Выведите топ-10 признаков по важности, согласно обученной модели;

In [38]:
features_train_upsampled, features_test, target_train_upsampled, target_test = train_test_split(
    features_ohe, target, test_size=0.25, random_state=23
)
model = LogisticRegressionCV(solver='liblinear', random_state=12, class_weight='balanced')
model.fit(features_train_upsampled, target_train_upsampled)
roc_auc_score(target_test, model.predict_proba(features_test)[:, 1])

0.7915212563451778

In [39]:
feature_importances = pd.DataFrame(
    zip(list(features_ohe.columns), list(abs(model.coef_[0]))), 
    columns=['feature', 'importance']
).sort_values(by=['importance'], ascending=False)

print('\nТоп-10 признаков по важности:')
print(feature_importances.head(10))


Топ-10 признаков по важности:
                                 feature  importance
24          Number.of.Successful.Grant.1    0.659722
25        Number.of.Unsuccessful.Grant.1    0.528271
269  Contract.Value.Band...see.note.A_A     0.470718
261              Grant.Category.Code_30B    0.333512
256              Grant.Category.Code_10A    0.247967
272  Contract.Value.Band...see.note.A_D     0.241869
275  Contract.Value.Band...see.note.A_G     0.215805
170                      Sponsor.Code_2B    0.182034
273  Contract.Value.Band...see.note.A_E     0.173888
274  Contract.Value.Band...see.note.A_F     0.132566


В модели логистической регрессии с использованием кросс-валидации ROC-AUC выше, чем в предыдущих примерах и близок к лучшему результату

Обучите модель случайного леса

Для подбора гиперпараметров и кросс-валидации используйте структуру GridSearchCV,

Выберите наилучший вариант случайного леса и выведите его параметры,

Оцените качество выбранной модели с помощью метрики rocauc,

Выведите топ-10 признаков по важности. Используйте атрибут feature_importances_, чтобы узнать важность признаков в деревянных моделях.

In [40]:
# Определение диапазона гиперпараметров для поиска
param_grid = {
    'n_estimators': [50]
}

# Создание экземпляра модели случайного леса и GridSearchCV
rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rf, param_grid=param_grid, cv=5, scoring='roc_auc')

# Обучение модели с помощью GridSearchCV
grid_search.fit(features_train_sc, target_train)

# Создание и обучение модели с лучшими гиперпараметрами
best_rf = RandomForestClassifier(random_state=42, **grid_search.best_params_)
best_rf.fit(features_train_sc, target_train)

# Оценка качества модели с помощью метрики ROC AUC
roc_auc = roc_auc_score(target_test, best_rf.predict_proba(features_valid_sc)[:, 1])
print('ROC AUC score:', roc_auc)

# Вывод топ-10 признаков по важности
feature_importances = pd.DataFrame(
    zip(list(features_ohe.columns), best_rf.feature_importances_),
    columns=['feature', 'importance']
).sort_values(by=['importance'], ascending=False)

print('\nТоп-10 признаков по важности:')
print(feature_importances.head(10))


ROC AUC score: 0.8693289974619289

Топ-10 признаков по важности:
                                 feature  importance
25        Number.of.Unsuccessful.Grant.1    0.115346
10                            SEO.Code.1    0.041991
22                            Dept.No..1    0.041321
24          Number.of.Successful.Grant.1    0.040495
0                            RFCD.Code.1    0.039849
20                           Person.ID.1    0.039580
27                                   A.1    0.029704
28                                   B.1    0.029563
21                       Year.of.Birth.1    0.028197
269  Contract.Value.Band...see.note.A_A     0.027404


5. _Напишите вывод, в котором будет содержаться информация о том, какие признаки важны согласно обеим моделям, и какая модель оказалась наилучшей для решения поставленной задачи._

Согласно представленному анализу, можно сделать следующие выводы:

 Ключевые признаки, важные для обеих моделей:
   - Number.of.Unsuccessful.Grant.1 - количество неуспешных заявок на гранты
   - Number.of.Successful.Grant.1 - количество успешных заявок на гранты
   - Contract.Value.Band...see.note.A_Unknown - неизвестный диапазон стоимости контракта

Эти признаки, по-видимому, являются наиболее значимыми предикторами успешности заявок на гранты в данных.

 Сравнение моделей:
   - Модель логистической регрессии показала ROC AUC score 0.76, что является достаточно хорошим результатом.
   - Модель случайного леса продемонстрировала более высокий ROC AUC score 0.87, что указывает на ее лучшую производительность по сравнению с логистической регрессией.

Таким образом, модель случайного леса оказалась наилучшей для решения поставленной задачи. Она смогла достичь более высокой точности предсказания успешности заявок на гранты, вероятно, за счет того, что случайный лес лучше справляется с нелинейными зависимостями и взаимодействиями между признаками по сравнению с линейной моделью логистической регрессии.

Ключевыми факторами, влияющими на успешность заявок на гранты, согласно обеим моделям, являются количество как успешных, так и неуспешных прошлых заявок, а также неизвестный диапазон стоимости контракта. Эта информация может быть полезна для лучшего понимания процесса получения грантов и принятия более обоснованных решений.

___Задание ПРО___

Основные цели этого задания:

Познакомиться с фреймворком CatBoost

Научиться готовить данные для обучения с помощью него

Попрактиковаться в работе с CatBoost

Задача: по различным признакам, связанным с заявкой на грант, предсказать, будет ли заявка принята, используя для обучения модели фреймворк CatBoost.

Обучите модель с помощью библиотеки CatBoost. Узнать, как называются параметры, нужные для настройки модели, можно на странице документации, посвященной классификатору. Подберите такие гиперпараметры модели, чтобы ее качество (по метрике RocAuc) на валидационной части было не ниже, а лучше и выше качества прогноза, полученного Вами в обычной версии задания, с помощью самостоятельной предобработки данных (от 0.9).

Напишите вывод, в котором сформулируйте свои наблюдения относительно качества, а также времени, необходимом для предобработки данных и обучении модели.

In [41]:
# Создание и обучение модели CatBoost
model = CatBoostClassifier(n_estimators=500, max_depth=2, learning_rate=0.2, random_state=42)
model.fit(features_train, target_train, eval_set=[(features_test, target_test)], early_stopping_rounds=100)

# Оценка качества модели
train_accuracy = balanced_accuracy_score(model.predict(features_train), target_train)
test_accuracy = balanced_accuracy_score(model.predict(features_test), target_test)
test_roc_auc = roc_auc_score(target_test, model.predict_proba(features_test)[:, 1])

print(f'Точность на обучающей выборке: {train_accuracy:.4f}')
print(f'Точность на тестовой выборке: {test_accuracy:.4f}')
print(f'ROC AUC score: {test_roc_auc:.4f}')

# Визуализация процесса обучения
evals_result = model.get_evals_result()
train_scores = evals_result['learn']
test_scores = evals_result['validation']
epochs = len(train_scores['Logloss'])
x_axis = list(range(epochs))

fig = go.Figure()
fig.add_trace(go.Scatter(x=x_axis, y=train_scores['Logloss'], mode='lines', name='Train'))
fig.add_trace(go.Scatter(x=x_axis, y=test_scores['Logloss'], mode='lines', name='Test'))

fig.update_layout(
    title='CatBoost Logloss',
    xaxis_title='Boosting Iterations',
    yaxis_title='Logloss',
    legend_title_text='Dataset'
)
fig.show()

0:	learn: 0.5892110	test: 0.5890831	best: 0.5890831 (0)	total: 95ms	remaining: 47.4s
1:	learn: 0.5049216	test: 0.5063235	best: 0.5063235 (1)	total: 97ms	remaining: 24.1s
2:	learn: 0.4708814	test: 0.4788788	best: 0.4788788 (2)	total: 99.7ms	remaining: 16.5s
3:	learn: 0.4403865	test: 0.4567542	best: 0.4567542 (3)	total: 102ms	remaining: 12.7s
4:	learn: 0.4226560	test: 0.4446922	best: 0.4446922 (4)	total: 106ms	remaining: 10.5s
5:	learn: 0.4087976	test: 0.4318215	best: 0.4318215 (5)	total: 108ms	remaining: 8.91s
6:	learn: 0.4004681	test: 0.4246140	best: 0.4246140 (6)	total: 110ms	remaining: 7.74s
7:	learn: 0.3955390	test: 0.4219120	best: 0.4219120 (7)	total: 112ms	remaining: 6.88s
8:	learn: 0.3878355	test: 0.4171936	best: 0.4171936 (8)	total: 114ms	remaining: 6.22s
9:	learn: 0.3827196	test: 0.4112955	best: 0.4112955 (9)	total: 116ms	remaining: 5.67s
10:	learn: 0.3744792	test: 0.4029246	best: 0.4029246 (10)	total: 118ms	remaining: 5.23s
11:	learn: 0.3704294	test: 0.3993158	best: 0.3993158 

Точность на обучающей выборке: 0.9270
Точность на тестовой выборке: 0.8251
ROC AUC score: 0.9136

Эти метрики выше, чем в предыдущем варианте задания, где вы использовали логистическую регрессию. Это показывает, что CatBoost справился с задачей лучше.

Обучение модели CatBoost может быть быстрее, чем ручная предобработка данных и обучение логистической регрессии. CatBoost автоматически обрабатывает категориальные признаки и может находить сложные взаимосвязи в данных, что существенно экономит время.